# Getting data from Citrination

In [1]:
from citrination_client import CitrinationClient
from citrination_client import PifQuery
from pypif.pif import dumps
import json 
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import pickle

In [3]:
client = CitrinationClient(site='my_site',api_key='my_key' ) 

In [4]:
list_of_datasets = [1,15,16]

In [5]:
# return list of all ids and 
# a dictionary where keys are ids of samples, values are the pandas dataframes
# with two columns - 'q' and 'I'
def get_data_from_Citrination(client, dataset_id_list):
    all_samples_names = [] # list of all id only - we can use it for random access
    data = {} # dict where keys are ids of samples
    for dataset in dataset_id_list:
        query_dataset = PifQuery(include_datasets=[dataset])
        query_result = client.search(query_dataset)
        pifs = [x.system for x in query_result.hits]
        for line in pifs:
            my_str = dumps(line)
            obj = json.loads(my_str) # to transform the string to dictionary
            for pr in obj['properties']:
                if pr['name'] == 'SAXS intensity':
                    q_list_of_dic = pr['conditions'][0]['scalars'] # q
                    q_list = []
                    for i in q_list_of_dic:
                        for k, v in i.items():
                            q_list.append(v)
                    I_list_of_dic = pr['scalars']# I
                    I_list = []
                    for i in I_list_of_dic:
                        for k, v in i.items():
                            I_list.append(v)
                    if (len(q_list) != 560 or len(I_list) != 560):
                        continue
                    sample_id = "set_" + str(dataset) + "_" + obj['uid']
                    all_samples_names.append(sample_id)
                    df = pd.DataFrame.from_dict({'q': q_list, 'I' : I_list})
                    df = df.astype(float)
                    data[sample_id] = df
    return all_samples_names, data

In [6]:
'''
Features: 'name', 'q_Imax', 'Imax_over_Imean','Imax_over_Imean_local', 'fluctuation_strength', 'low_q_ratio',
                 'high_q_ratio', 'Imax_over_Ilowq','Imax_over_Ihighq', 'Ilowq_over_Ihighq', 'b_s_1', 'b_s_2','b_s_3',
                 'b_s_4', 'b_s_5', 'b_s_6', 'b_s_7', 'b_s_8', 'b_s_9', 'b_s_10','b_s_11', 'b_s_12','b_s_13',
                 'b_s_14', 'b_s_15', 'b_s_16', 'b_s_17', 'b_s_18', 'b_s_19', 'b_s_20'
'''
def extract_features(df, name):
    features = []
    q = np.array(df['q'])
    I = np.array(df['I'])
    
    idxmax = np.argmax(I)
    Imax = I[idxmax]
    q_Imax = q[idxmax]
    
    idxmin = np.argmin(I)
    Imin = I[idxmin]
    Irange = Imax - Imin
    Imean = np.mean(I)
    Imax_over_Imean = float(Imax)/float(Imean)
    
    idx_around_max = ((q > 0.9*q_Imax) & (q < 1.1*q_Imax))
    Imean_around_max = np.mean(I[idx_around_max])
    Imax_over_Imean_local = Imax / Imean_around_max
    
    ### fluctuation analysis
    # array of the difference between neighboring points:
    nn_diff = I[1:]-I[:-1]
    # keep indices where the sign of this difference changes.
    # also keep first index
    nn_diff_prod = nn_diff[1:]*nn_diff[:-1]
    idx_keep = np.hstack((np.array([True]),nn_diff_prod<0))
    fluc = np.sum(np.abs(nn_diff[idx_keep]))
    fluctuation_strength = fluc/Imean
    
    I_sum = np.sum(I)
    low_q_ratio = np.sum(I[(q<0.4)])/I_sum
    high_q_ratio = np.sum(I[(q>=0.4)])/I_sum
    
    ### curve shape analysis
    lowq_idx = q<0.1
    highq_idx = q>0.4
    lowq = q[lowq_idx]
    highq = q[highq_idx]
    I_lowq = I[lowq_idx]
    I_highq = I[highq_idx]
    I_lowq_mean = np.mean(I_lowq)
    I_highq_mean = np.mean(I_highq)
    Imax_over_Ilowq = float(Imax)/I_lowq_mean
    Ilowq_over_Ihighq = I_lowq_mean/I_highq_mean
    Imax_over_Ihighq = float(Imax)/I_highq_mean
    
    bin_strengths = np.zeros(20)
    for i in range(20):
            qmini, qmaxi = i*0.05, (i+1)*0.05
            idxi = ((q>=qmini) & (q<qmaxi))
            if any(idxi):
                qi = q[ idxi ]
                Ii = I[ idxi ]/Imax # /Imax added
                dqi = qi[1:]-qi[:-1]
                Ii = (Ii[1:]+Ii[:-1])/2
                bin_strengths[i] = np.sum(np.log(Ii) * dqi) / (qi[-1]-qi[0])
  

    features.append(name)
    features.append(q_Imax)
    features.append(Imax_over_Imean)
    features.append(Imax_over_Imean_local)
    features.append(fluctuation_strength)
    features.append(low_q_ratio)
    features.append(high_q_ratio)
    features.append(Imax_over_Ilowq)
    features.append(Imax_over_Ihighq)
    features.append(Ilowq_over_Ihighq)
    
    for s in bin_strengths:
        features.append(s)
                  
    return tuple(features)

In [7]:
def create_data_frame(data, names):
    data_set = [] 

    columns = ['name', 'q_Imax', 'Imax_over_Imean','Imax_over_Imean_local', 'fluctuation_strength', 'low_q_ratio',
                 'high_q_ratio', 'Imax_over_Ilowq','Imax_over_Ihighq', 'Ilowq_over_Ihighq', 'b_s_1', 'b_s_2','b_s_3',
                 'b_s_4', 'b_s_5', 'b_s_6', 'b_s_7', 'b_s_8', 'b_s_9', 'b_s_10','b_s_11', 'b_s_12','b_s_13',
                 'b_s_14', 'b_s_15', 'b_s_16', 'b_s_17', 'b_s_18', 'b_s_19', 'b_s_20' ]
    for f in names:
        df = data[f]
        data_set.append(extract_features(df, f))
    data_frame = pd.DataFrame.from_records(data_set, columns=columns)
    return data_frame

In [28]:
'''
Features: whole q array
'''
def extract_features2(df, name):
    features = []
    i = np.array(df['I'])
    features.append(name)
    for s in i:
        features.append(s)           
    return tuple(features)

In [24]:
# using whole q array as features
def create_data_frame2(data, names):
    data_set = [] 
    columns = list(range(0, 560))
    columns = list(map(str, columns))
    columns.insert( 0, 'name')

    for f in names:
        df = data[f]
        data_set.append(extract_features2(df, f))
    data_frame = pd.DataFrame.from_records(data_set, columns=columns)
    return data_frame

## get_data_and_create_df(client, dataset_id_list)

In [21]:
from sklearn.cluster import MiniBatchKMeans
from sklearn import preprocessing

def create_and_save_model(client, dataset_id_list):
    names, data = get_data_from_Citrination(client, dataset_id_list)
    df = create_data_frame(data, names)
    
    scaler = preprocessing.StandardScaler()
    scaler.fit(df[features])
    # put my best unsupervised model here
    # for example:
    # clusterer = MiniBatchKMeans(n_clusters=n_clusters, random_state=10)
    # clusterer.fit(scaler.transform(df[features]))
    #model.to_pickle('unsupervised_model.pkl')

In [22]:
# it is a temporary function for testing
def get_data_and_create_df(client, dataset_id_list):
    names, data = get_data_from_Citrination(client, dataset_id_list)
    df = create_data_frame(data, names)
    return df

def get_data_and_create_df2(client, dataset_id_list):
    names, data = get_data_from_Citrination(client, dataset_id_list)
    df = create_data_frame2(data, names)
    return df

In [327]:
data_frame = get_data_and_create_df(client, list_of_datasets)

In [29]:
# with q as features
data_frame2 = get_data_and_create_df2(client, list_of_datasets)
data_frame2.head(3)

,name,0,1,2,3,4,5,6,7,8,...,550,551,552,553,554,555,556,557,558,559
0,set_1_R1_1479582658,1.143534,0.497115,0.000000,0.473436,0.487803,0.906999,1.823430,1.289826,1.694177,...,2.251870,2.112009,2.084345,2.232733,2.299173,2.220341,2.055935,2.028783,2.046827,2.168071
1,set_1_R1_1479582669,0.000000,1.135393,1.251626,0.702255,0.592320,0.419255,1.134044,1.444593,1.232273,...,1.831821,2.005809,1.915149,1.888664,1.899058,1.803673,1.824841,1.802742,1.857838,1.863579
2,set_1_R1_1479582681,0.021854,0.157678,0.295437,0.000000,0.678777,0.898697,0.880770,1.121462,1.715802,...,1.793492,1.713437,1.781256,1.888325,1.836691,1.850607,1.762937,1.877357,1.942171,1.954817


In [30]:
data_frame2.shape

(2170, 561)

In [331]:
#save the dataframe with analytical features and bins
data_frame.to_pickle('df_for_clustering.pkl')

In [33]:
#save the dataframe with whole q array as features
data_frame2.to_pickle('df_for_clustering2.pkl')